In [1]:
! pip install  ../

Defaulting to user installation because normal site-packages is not writeable
Processing /home/briarmoss/Documents/Boston_311
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for boston311: filename=boston311-0.2.0-py3-none-any.whl size=23980 sha256=89136a3c02373d9e07b9ff81c09e38aa4aa40f764a88ad70a7b0018c4fbfe49d
  Stored in directory: /tmp/pip-ephem-wheel-cache-i6pcy3a_/wheels/3d/69/ee/0a6ac96b9c09c948fc0e74f2724a9703aa39749a41fa757c9e
Successfully built boston311
  Attempting uninstall: boston311
    Found existing installation: boston311 0.2.0
    Uninstalling boston311-0.2.0:
      Successfully uninstalled boston311-0.2.0


In [2]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from kerastuner import HyperParameters
from boston311 import Boston311LogReg, Boston311KerasNN

Using TensorFlow backend


2023-10-31 23:10:50.946645: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-31 23:10:51.196135: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-31 23:10:51.197699: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-31 23:10:51.878190: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/tmp/ipykernel_505058/1626478388.py:5: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import HyperParameters


In [3]:
today_datestring, tomorrow_datestring, thirty_days_ago_datestring = Boston311LogReg().get_datestrings()

In [4]:
#set model folder constant
MODEL_FOLDER = './daily_models'

In [5]:
EXTRA_mydata_FILE = './cls_and_pooled_embeddings_with_three_cols.csv'

In [6]:
pkl_load_data = Boston311LogReg().pkl_load_data

In [7]:
KerasNN_model = Boston311KerasNN(train_date_range={'start':'2023-01-01','end':thirty_days_ago_datestring},
                            predict_date_range={'start':thirty_days_ago_datestring,'end':today_datestring},
                            feature_columns=['queue', 'subject', 'reason', 'department'],
                            scenario={'dropColumnValues': {'source':['City Worker App', 'Employee Generated']},
                                      'survivalTimeMin':300,
                                      'survivalTimeFill':tomorrow_datestring})

In [8]:
#get current datetime in Boston timezone as string
my_datetime_str = Boston311LogReg().get_current_datetime_str()

In [9]:
case_data_file = 'case_data.pkl'
case_data_csv = 'all_311_cases.csv'
data = pkl_load_data(case_data_csv, case_data_file)
mydata = KerasNN_model.load_data(data)

In [10]:
mydata = KerasNN_model.enhance_data(mydata)

In [11]:
mydata = KerasNN_model.apply_scenario(mydata)

In [12]:
mydata = KerasNN_model.clean_data(mydata)

In [13]:
from ast import literal_eval
import pickle

pickle_file = 'dataframe.pkl'

df = None

df = pkl_load_data(EXTRA_mydata_FILE, pickle_file)

# if df has a column service_request_id, do the following
if 'service_request_id' in df.columns:
    df.rename(columns={'service_request_id':'case_enquiry_id'}, inplace=True)
    for col in ['desc_cls_embedding', 'desc_pooled_embedding', 'name_cls_embedding', 'name_pooled_embedding', 'code_cls_embedding', 'code_pooled_embedding']:
        df[col] = df[col].apply(literal_eval).apply(np.array)

    pickle.dump(df, open(pickle_file, "wb"))

In [14]:
def flatten_and_replace_columns(df, column_names):
    new_dfs = []
    
    # Flatten and remove original columns
    for col_name in column_names:
        flattened = np.stack(df[col_name].to_numpy())
        new_df = pd.DataFrame(flattened, columns=[f'{col_name}_{i}' for i in range(flattened.shape[1])])
        new_dfs.append(new_df)
        df.drop([col_name], axis=1, inplace=True)
        
    # Concatenate new columns to the original DataFrame
    df = pd.concat([df] + new_dfs, axis=1)
    return df

# Assuming df is your DataFrame
column_names = ['desc_cls_embedding', 'desc_pooled_embedding', 'name_cls_embedding', 'name_pooled_embedding', 'code_cls_embedding', 'code_pooled_embedding']
df = flatten_and_replace_columns(df, column_names)

In [15]:
df['case_enquiry_id'] = df['case_enquiry_id'].astype(str)
is_numeric = df['case_enquiry_id'].str.isnumeric()
df = df[is_numeric]

In [16]:
df['case_enquiry_id'] = df['case_enquiry_id'].astype('int64')

In [17]:
df = df.drop_duplicates(subset=['case_enquiry_id']) 

In [19]:
mydata = mydata.drop_duplicates(subset=['case_enquiry_id'])

In [20]:
#join them so we are left only with records that have mydata in both files
new_mydata = mydata.merge(df, on='case_enquiry_id', how='inner')

In [21]:
new_mydata = new_mydata.sort_values(by='case_enquiry_id')

In [22]:
hour_interval = 72
max_days = 180
#bin_edges = KerasNN_model.generate_time_bins_statistics(df, num_intervals=60)
bin_edges= KerasNN_model.generate_time_bins_fixed_interval(hour_interval, max_days)
bin_labels= KerasNN_model.generate_bin_labels(bin_edges)
bin_number = len(bin_labels)

In [23]:
df, y = KerasNN_model.split_data(new_mydata, bin_edges=bin_edges, bin_labels=bin_labels)

In [24]:
#cast all columns that are type bool to float
for col in df.columns:
    if df[col].dtype == 'bool':
        df[col] = df[col].astype('float64')

In [25]:
#best_model, best_hyperparameters = KerasNN_model.tune_model(df, y, '/home/briarmoss/Documents/Boston_311/models/tuning')

In [26]:
#set constants
start_nodes = 1024  
end_nodes = 128
#l2_0 = 0.00001
#learning_rate = 7.5842e-05
l2_0 = 0.001
learning_rate = 0.0001


hp = HyperParameters()
hp.Fixed('start_nodes', start_nodes)
hp.Fixed('end_nodes', end_nodes)
hp.Fixed('l2_0', l2_0)
hp.Fixed('learning_rate', learning_rate)
hp.Fixed('final_layer', bin_number)
hp.Fixed('final_activation', 'softmax')
KerasNN_model.best_hyperparameters = hp

In [29]:

#parse CLS embedding column as array
test_acc = KerasNN_model.train_model( df, y , epochs=2)

Starting Training at 2023-10-31 23:11:12.240150
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1024)              1033216   
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                                 
 dense_3 (Dense)             (None, 128)               32896     
                                                                 
 dense_4 (Dense)             (None, 61)                7869      
                                                                 
Total params: 1730109 (6.60 MB)
Trainable params: 1730109 (6.60 MB)
Non-trainable params: 0 (0.00 Byte)
____________________________________

2023-10-31 23:11:12.937621: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 761798016 exceeds 10% of free system memory.


739/739 [==============================] - 10s 13ms/step - loss: 2.6450 - accuracy: 0.7086 - top_k_categorical_accuracy: 0.7924 - val_loss: 2.0150 - val_accuracy: 0.7277 - val_top_k_categorical_accuracy: 0.8174
Epoch 2/2
739/739 [==============================] - 1s 1ms/step - loss: 1.8118 - accuracy: 0.7155 - top_k_categorical_accuracy: 0.8040
Testing accuracy: 0.7155135869979858 
Top-2 accuracy: 0.8039630651473999 
Test loss: 1.8117965459823608
Ending Training at 2023-10-31 23:11:34.101699
Training took 0:00:21.861549


In [30]:

def save_model_to_dir(model, folder_name):
    dir_path = os.path.join(MODEL_FOLDER, folder_name)
    
    if not os.path.exists(dir_path):
        os.mkdir(dir_path)
    
    timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    model_name = timestamp + "_" + model.model_type
    properties_name = model_name
    
    model.save(dir_path, model_name, properties_name)


save_model_to_dir(KerasNN_model, KerasNN_model.model_type)


/home/briarmoss/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [31]:
"""
data = KerasNN_model.load_data( 'predict' )
data = KerasNN_model.enhance_data( data, 'predict')
clean_data = KerasNN_model.clean_data_for_prediction( data )

X_predict, y_predict = KerasNN_model.split_data( clean_data )
y_predict = KerasNN_model.model.predict(X_predict)
data['survival_prediction'] = y_predict
return data
"""


"\ndata = KerasNN_model.load_data( 'predict' )\ndata = KerasNN_model.enhance_data( data, 'predict')\nclean_data = KerasNN_model.clean_data_for_prediction( data )\n\nX_predict, y_predict = KerasNN_model.split_data( clean_data )\ny_predict = KerasNN_model.model.predict(X_predict)\ndata['survival_prediction'] = y_predict\nreturn data\n"